In [1]:
import pandas_gbq
import pydata_google_auth

SCOPES = [
    'https://www.googleapis.com/auth/cloud-platform',
    'https://www.googleapis.com/auth/bigquery'
]

credentials = pydata_google_auth.get_user_credentials(
    SCOPES, 
    use_local_webserver=True
)

PROJECT_ID = "sta-track-sql" # 여러분들의 구글클라우드 프로젝트 ID

# Order items 기준으로 <- products,orders <- users로 join
query = """
SELECT *
FROM ecommerce_advanced.order_items
LEFT JOIN ecommerce_advanced.products
USING(product_id)
LEFT JOIN ecommerce_advanced.orders
USING(order_id)
LEFT JOIN ecommerce_advanced.users
USING(user_id)
"""

df = pandas_gbq.read_gbq(query_or_table=query, project_id=PROJECT_ID, credentials=credentials)
df.info()

Downloading: 100%|██████████|
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180 entries, 0 to 179
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype              
---  ------             --------------  -----              
 0   user_id            180 non-null    Int64              
 1   order_id           180 non-null    Int64              
 2   product_id         180 non-null    Int64              
 3   order_item_id      180 non-null    Int64              
 4   quantity           180 non-null    Int64              
 5   price_per_unit     180 non-null    object             
 6   discount           180 non-null    object             
 7   product_name       180 non-null    object             
 8   category_id        180 non-null    Int64              
 9   price              180 non-null    object             
 10  stock_quantity     180 non-null    Int64              
 11  added_date         180 non-null    dbdate             
 12  supplier_id        1

## RFM 1 Pager 작성하기

- Recency (R): 고객이 얼마나 최근에 구매했는가?

- Frequency (F): 고객이 얼마나 자주 구매했는가?

- Monetary (M): 고객이 얼마나 많은 돈을 썼는가?


### 직접 ERD 작성하고 필요한 데이터 추출하기
![erd](ecommerce_ERD_image.png)

### 프로젝트의 목표 : 해당 유저의 유형이 올바르게 RFM에 의해서 군집이 되었는지에 대한 의문
    - 현재 30명의 user는 Regular, Premium의 2개의 등급을 보유 중
    - 이 등급이 구매 패턴에 의해 형성된 것이 맞는지, 임의로 형성된 값인지, 2등급이 아닌 5개 이상의 등급으로 나눌 순 없는지에 대한 분석을 진행함

따라서, 회원정보, 구매 내역, 구매 상세 내역에서 날짜 및 거래 정보를 통해 최근 구매, 자주 구매 패턴을 확인한다.

결론: 총 7개의 테이블 중 **users, products, orders, order_items**에서 RFM 지표를 뽑아낸다.


- SQL에서는 JOIN만 진행하고 나머지 데이터 전처리는 Pandas를 활용한다.

In [2]:
df.order_date.describe()

count                                 180
mean     2023-09-09 12:28:20.250000+00:00
min             2023-01-05 14:32:15+00:00
25%      2023-05-11 15:10:48.750000+00:00
50%      2023-09-07 12:00:27.500000+00:00
75%             2024-01-07 09:30:33+00:00
max             2024-05-18 12:15:30+00:00
Name: order_date, dtype: object

## 1. Recent 처리하기
- 주문 일자 기준으로는 23년 1월 5일부터 24년 5월 18일까지의 데이터가 있음
- 고객별로 가장 마지막에 주문한(최근에 주문한) 날짜를 계산하기
- 필요 컬럼: user_id, order_date => last_order_date 
- 24년 5월 18일 기준으로 언제 주문했는지 처리하기
## 2. Frequency 처리하기
- **고객이 얼마나 자주 우리 서비스를 이용하는지?**
- 충성도나 재방문율을 측정하는데 적합한 지표
- 주문 건수? 구매량 수? 고민해보기
    1. 주문한 건수로 보는게 맞을지 주문 수량으로 보는게 맞을지
    2. 근데 제품이 다 다른데 어떻게 기준을 잡으면 좋을까?
=> 일반적인 RFM에선 주문 건수로 보는 것이 표준이고, 구매량 수는 빈도보단 규모
- 딱 한 번의 대량 주문으로 F 수치가 높아질 가능성 존재하기 때문에 **주문 건수**로 확인한다
- 필요 컬럼 : 회원당 주문 건수 COUNT


## 3. Monument 처리하기
- (단위당 지출액 - 단위당 할인가) * 주문량 = 주문당 지출액 구하기
- 고객별 총 지출액을 SUM해서 확인해보기

In [3]:
import pandas as pd
import plotly.express as px

rfm_df = pd.DataFrame(index= df['user_id'].unique())
rfm_df.index.name = 'user_id'
rfm_df['prev_grade'] = df['user_type']
rfm_df.head() # 기존 등급이랑 RFM 이후 등급 확인해보기


,prev_grade
user_id,
12,REGULAR
26,PREMIUM
27,REGULAR
1,REGULAR
9,REGULAR


In [4]:
rfm_df['last_order_date'] = df.groupby('user_id')['order_date'].max().dt.date
rfm_df.last_order_date[:5]

user_id
12    2024-05-16
26    2024-03-19
27    2024-01-23
1     2024-03-31
9     2024-04-16
Name: last_order_date, dtype: object

In [5]:
# 기준 날짜 설정
standard_date = pd.to_datetime('2024-05-19')
rfm_df['Recency'] = (standard_date - pd.to_datetime(rfm_df['last_order_date']).dt.tz_localize(None)).dt.days
rfm_df.head()

,prev_grade,last_order_date,Recency
user_id,,,
12,REGULAR,2024-05-16,3
26,PREMIUM,2024-03-19,61
27,REGULAR,2024-01-23,117
1,REGULAR,2024-03-31,49
9,REGULAR,2024-04-16,33


In [6]:
# Frequency 계산 (주문 건수)
rfm_df['Frequency'] = df.groupby('user_id')['order_id'].count()
rfm_df.head()


,prev_grade,last_order_date,Recency,Frequency
user_id,,,,
12,REGULAR,2024-05-16,3,9
26,PREMIUM,2024-03-19,61,3
27,REGULAR,2024-01-23,117,5
1,REGULAR,2024-03-31,49,7
9,REGULAR,2024-04-16,33,6


In [7]:
# Monetary 계산 (모든 제품 카테고리 구매액 합산)
monetary_cols = ['price_per_unit','discount','quantity']
df['total_revenue_per_order'] =(df['price_per_unit']-df['discount']) * df['quantity'] 

rfm_df['Monetary']= df.groupby('user_id')['total_revenue_per_order'].sum().astype(int)
rfm_df.head()

,prev_grade,last_order_date,Recency,Frequency,Monetary
user_id,,,,,
12,REGULAR,2024-05-16,3,9,3522000
26,PREMIUM,2024-03-19,61,3,2297000
27,REGULAR,2024-01-23,117,5,1209000
1,REGULAR,2024-03-31,49,7,5592000
9,REGULAR,2024-04-16,33,6,6443000


### rfm_df 분포 확인해보기
- 분포를 확인해서 어떤 스코어링 방식을 사용할지 확인한다.

In [8]:
pd.options.display.float_format = '{:.4f}'.format

In [9]:
rfm_df[['Recency', 'Frequency', 'Monetary']].describe()

,Recency,Frequency,Monetary
count,30.0000,30.0000,30.0000
mean,55.4667,6.0000,2094433.3333
std,44.9580,2.7792,2221930.7336
min,1.0000,2.0000,217000.0000
25%,18.0000,3.2500,683500.0000
50%,47.0000,6.0000,1243000.0000
75%,76.0000,7.7500,2471750.0000
max,137.0000,12.0000,9452000.0000


분포 분석해보기
1. recentcy : 25%,50%,75%가 각각 18, 47, 76으로 분포
2. frequency : frequency가 큰 차이를 보이지 않고 골고루 분포됨
3. monetary : 평균이 약 210만원인데, 최소값이 20만원, 중앙값이 124만원 최댓값이 945만원 수준으로 편차가 매우 큰 것을 확인할 수 있다.  
이는 소수 고객이 구매력이 매우 커서 평균을 높이는 것으로 확인할 수 있다.

In [10]:
rfm_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 30 entries, 12 to 28
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   prev_grade       30 non-null     object
 1   last_order_date  30 non-null     object
 2   Recency          30 non-null     int64 
 3   Frequency        30 non-null     Int64 
 4   Monetary         30 non-null     int32 
dtypes: Int64(1), int32(1), int64(1), object(2)
memory usage: 1.3+ KB


- astype 변경하기(Int64로 통일)

In [11]:
rfm_df = rfm_df.astype({
    'Recency': 'Int64',
    'Frequency': 'Int64',
    'Monetary': 'Int64'
})
rfm_df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 30 entries, 12 to 28
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   prev_grade       30 non-null     object
 1   last_order_date  30 non-null     object
 2   Recency          30 non-null     Int64 
 3   Frequency        30 non-null     Int64 
 4   Monetary         30 non-null     Int64 
dtypes: Int64(3), object(2)
memory usage: 1.5+ KB


### RFM 스코어 부여 및 세그먼트 정의하기

- 분포가 다른 R, F, M 지표들을 공정한 기준으로 처리하기 위해 점수로 변환하기
- 분위수, 균등 간격 분할, 수동 규칙 기반 중 Monetary의 값이 한 쪽으로 쏠려있기 때문에 오분위수(q=5)로 판단한다.
- RFM 값을 점수로 변환하는 과정 대신 K-means 클러스터링 분석도 진행할 수 있으나 과제엔 RFM Score에 집중한다.


**점수 부여 규칙:**
-   **Recency (R)**: 값이 **작을수록** (더 최근일수록) 좋은 것이므로, 가장 높은 점수(5점)를 부여합니다.
-   **Frequency (F), Monetary (M)**: 값이 **클수록** 좋은 것이므로, 가장 높은 점수(5점)를 부여합니다.

In [12]:
r_labels = range(4, 0, -1) # 작은 것이 좋은 것
f_labels = range(1, 5) # 큰 것이 좋은 것
m_labels = range(1, 5) # 큰 것이 좋은 것

# qcut 분위수로 잘라서 등급(점수) 매기기
rfm_df['R_Score'] = pd.qcut(rfm_df['Recency'], q=4, labels=r_labels, duplicates='drop')
rfm_df['F_Score'] = pd.qcut(rfm_df['Frequency'], q=4, labels=f_labels, duplicates='drop')
rfm_df['M_Score'] = pd.qcut(rfm_df['Monetary'], q=4, labels=m_labels, duplicates='drop')

rfm_df['RFM_Score'] = rfm_df['R_Score'].astype(int) + rfm_df['F_Score'].astype(int) + rfm_df['M_Score'].astype(int)
rfm_df.head()

,prev_grade,last_order_date,Recency,Frequency,Monetary,R_Score,F_Score,M_Score,RFM_Score
user_id,,,,,,,,,
12,REGULAR,2024-05-16,3,9,3522000,4,4,4,12
26,PREMIUM,2024-03-19,61,3,2297000,2,1,3,6
27,REGULAR,2024-01-23,117,5,1209000,1,2,2,5
1,REGULAR,2024-03-31,49,7,5592000,2,3,4,9
9,REGULAR,2024-04-16,33,6,6443000,3,2,4,9


## 총 2가지의 segment 비교 진행

1. 가장 기본적인 score 합산 기반 고객 세그먼트 `RFM_Segment1`
    -   score_bins = [0,6,9,12,15]
    -   score_labels = ['Bronze', 'Silver', 'Gold', 'Platinum']
    - **매우 간단하고 직관적**, 명확한 등급 나누기가 편함
    - 그러나 RFM 각각의 개별 특성이 뭉개질 수 있어서 주의



#### 2. R,F,M 각각의 지표를 확인한 4가지 핵심 그룹으로 압축해서 특성을 구분 짓는 고객 세그먼트 `RFM_Segment2`  

- 조건은 사분위수로 나타냄, 8분할 후 분포에 따라 합칠 세그먼트 확인할 것

    | 세그먼트명   | 조건| 설명 |
    |--------------|-----------------------------------------|--------------------------------------------------------------|
    | VIP          | r ≥ 3, f = 4, m = 4                    | 모든 지표에서 상위 25% 이상의 비율을 차지한 고객              |
    | Core         | r ≥ 3, f ≥ 2, m ≥ 3 (단, VIP 제외)      | 비즈니스의 핵심, 반드시 유지하고 특별 관리가 필요한 고객      |
    | Frequent     | r ≥ 2, f ≥ 3, m ≤ 2                    | 구매 방문이 최근이면서 자주 주문하지만 구매력이 낮은 고객     |
    | Past Vip     | r = 1, m ≥ 3                           | 예전엔 많이 썼으나 최근엔 안 오는 고객                        |
    | At_Risk      | r ≤ 2, f ≥ 3, m ≥ 3 (단, Past Vip 제외)| 과거에 구매, 빈도는 높으나 최근에 뜸해진 고객                |
    | Newbie       | r = 4, f ≤ 2, m ≤ 2                    | 최근에 구매했지만 아직 구매빈도, 구매금액이 크지 않은 고객    |
    | Inactive     | r ≤ 2, f = 1, m ≤ 2                    | 방문도 오래됐고, 구매 빈도, 금액도 낮은 고객                  |
    | General      | 위 조건에 해당하지 않는 고객            | 평균적인 고객                                                |



In [13]:
# 세그먼트 1 진행(RFM 합산 스코어 기준 분배)
score_bins = [0,6,8,10,12]
score_labels = ['Bronze', 'Silver', 'Gold', 'Platinum']
rfm_df['RFM_Segment1'] = pd.cut(rfm_df['RFM_Score'], bins=score_bins, labels=score_labels)

print(rfm_df['RFM_Segment1'].value_counts())

RFM_Segment1
Bronze      14
Gold         9
Platinum     4
Silver       3
Name: count, dtype: int64


In [14]:
# 세그먼트 2 진행 
def target_segment(row):
    r = row['R_Score']
    f = row['F_Score']
    m = row['M_Score']
    if r >= 3 and f == 4 and m == 4:
        return 'Vip'
    elif r >= 3 and f >= 2 and m >= 3:
        return 'Core'
    elif r >= 2 and f >= 3 and m <= 2:
        return 'Frequent'
    elif r == 1 and m >= 3:
        return 'Past Vip'
    elif r <= 2 and f >= 3 and m >= 3:
        return 'At_Risk'
    elif r == 4 and f <= 2 and m <= 2:
        return 'Newbie'
    elif r <= 2 and f == 1 and m <= 2:
        return 'Inactive'
    else:
        return 'General'


rfm_df['RFM_Segment2'] = rfm_df.apply(target_segment, axis=1)
rfm_df.head()
print(rfm_df['RFM_Segment2'].value_counts())



RFM_Segment2
Core        8
General     5
Inactive    5
Frequent    4
Past Vip    3
Vip         2
Newbie      2
At_Risk     1
Name: count, dtype: int64


#### 3. 지나치게 많은 세그먼트 8개 -> 5개(VIP 제외)로 통합시키기 `RFM_Segment3`
- 30명의 데이터를 8개의 등급으로 나누는 것보단 5개(VIP 제외)로 나눠서 확인해보기

- 조건은 사분위수로 나타냄, 8분할 후 분포에 따라 합칠 세그먼트 확인할 것

    | 세그먼트명   | 조건| 설명 |
    |--------------|-----------------------------------------|--------------------------------------------------------------|
    | Vip	|최우수고객|	충성도와 구매력이 모두 높은 고객|
    | Core	|핵심 고객|	충성도와 구매력이 평균 이상 고객|
    |Frequent, Newbie|	잠재고객(Potential/Active)|	자주 오거나 최근에 온, 성장 가능성 있는 고객|
    |Past Vip, At_Risk|	이탈위험(At Risk)|	과거에 우수했으나 최근 뜸해진 고객|
    |Inactive	|비활성(Inactive)	|거의 활동 없는 고객|
    |General|	일반(General)	|평균적인 고객|

In [15]:
def merge_segment(seg):
    if seg  == 'Vip':
        return '최우수고객'
    elif seg  == 'Core':
        return '핵심고객'
    elif seg in ['Frequent', 'Newbie']:
        return '잠재고객'
    elif seg in ['Past Vip', 'At_Risk']:
        return '이탈위험'
    elif seg == 'Inactive':
        return '비활성'
    else:
        return '일반'

rfm_df['RFM_Segment3'] = rfm_df['RFM_Segment2'].apply(merge_segment)
print(rfm_df['RFM_Segment3'].value_counts())

RFM_Segment3
핵심고객     8
잠재고객     6
일반       5
비활성      5
이탈위험     4
최우수고객    2
Name: count, dtype: int64


In [16]:
# 세그먼트3 별 RFM 지표 평균값 분석
segment_summary = rfm_df.groupby('RFM_Segment3').agg({
    'Recency': ['mean', 'std'],
    'Frequency': ['mean', 'std'], 
    'Monetary': ['mean', 'std'],
    'RFM_Score': ['mean', 'count']
}).round(2)

segment_summary.columns = ['최근성_평균', '최근성_표준편차', '구매빈도_평균', '구매빈도_표준편차', 
                          '구매금액_평균', '구매금액_표준편차', 'RFM점수_평균', '고객수']
print("세그먼트3별 RFM 지표 요약:")
segment_summary

세그먼트3별 RFM 지표 요약:


,최근성_평균,최근성_표준편차,구매빈도_평균,구매빈도_표준편차,구매금액_평균,구매금액_표준편차,RFM점수_평균,고객수
RFM_Segment3,,,,,,,,
비활성,98.6000,29.2000,2.6000,0.5500,735000.0000,350230.6400,4.0000,5
이탈위험,99.0000,43.1400,4.2500,1.8900,3000750.0000,2050688.4000,6.5000,4
일반,77.0000,44.9900,4.6000,1.1400,1047200.0000,769540.9000,5.4000,5
잠재고객,28.5000,30.5400,7.3300,1.7500,530833.3300,240018.6800,7.6700,6
최우수고객,24.0000,29.7000,10.0000,1.4100,3026000.0000,701449.9300,11.5000,2
핵심고객,21.3800,12.7400,7.8800,2.3600,4085250.0000,2947054.5000,10.0000,8


In [17]:
# 세그먼트별 3D 산점도 시각화 (R, F, M 관계)
fig_3d = px.scatter_3d(rfm_df, x='Recency', y='Frequency', z='Monetary',
                       color='RFM_Segment3', size='RFM_Score',
                       title='RFM 세그먼트3 별 3차원 분포',
                       labels={'Recency': '최근성 (일)', 
                              'Frequency': '구매빈도 (회)', 
                              'Monetary': '구매금액 (원)'})
fig_3d.show()

#### 추가 분석 수행
- 기존 라벨링된 등급과 RFM 스코어 합산 기반 등급의 차이는 어느 정도 일까?

### 히트맵을 그려서 이전 등급과 현재 rfm score 등급을 비교해보기
- rfm 등급 
    - **0~5** -> bronze 
    - **6~8** -> silver 
    - **9~11** -> gold 
    - **12~15** -> platinum 

비교를 위해 bronze, silver를 regular와 비교, gold, platinum을 premium과 비교한다.

In [18]:
# 1. RFM 등급 그룹핑 컬럼 추가해서 이전 등급과 비교
def rfm_group(seg):
    if seg in ['Bronze', 'Silver']:
        return 'REGULAR'
    elif seg in ['Gold', 'Platinum']:
        return 'PREMIUM'

rfm_df['RFM_Group'] = rfm_df['RFM_Segment1'].apply(rfm_group)
cross_table_2x2 = pd.crosstab(rfm_df['RFM_Group'], rfm_df['prev_grade'])
print(cross_table_2x2)

prev_grade  PREMIUM  REGULAR
RFM_Group                   
PREMIUM           2       11
REGULAR          11        6


In [19]:
import plotly.graph_objects as go

fig = go.Figure(data=go.Heatmap(
    z=cross_table_2x2.values,
    x=cross_table_2x2.columns,
    y=cross_table_2x2.index,
    colorscale='RdBu',
    text=cross_table_2x2.values,
    texttemplate="%{text}",
    hoverongaps=False
))

fig.update_layout(
    title='RFM 그룹 vs 기존 등급 2x2 Heatmap',
    xaxis_title='기존 등급 (prev_grade)',
    yaxis_title='RFM 그룹',
    title_x=0.5,
    width=500,
    height=400
)

fig.show()

In [20]:
print(rfm_df['prev_grade'].value_counts())
print(rfm_df['RFM_Group'].value_counts())

prev_grade
REGULAR    17
PREMIUM    13
Name: count, dtype: int64
RFM_Group
REGULAR    17
PREMIUM    13
Name: count, dtype: int64


In [21]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

# 실제값, 예측값 준비
y_true = rfm_df['prev_grade']      # 이전 라벨링 된 등급
y_pred = rfm_df['RFM_Group']       # RFM 기반 등급

# confusion matrix
labels = ['REGULAR', 'PREMIUM']
cm = confusion_matrix(y_true, y_pred, labels=labels)
print("Confusion Matrix:\n", cm)

# classification report
print("\nClassification Report:")
print(classification_report(y_true, y_pred, labels=labels, target_names=labels))

# 정확도
acc = accuracy_score(y_true, y_pred)
print(f"\n정확도(Accuracy): {acc:.2f}")

Confusion Matrix:
 [[ 6 11]
 [11  2]]

Classification Report:
              precision    recall  f1-score   support

     REGULAR       0.35      0.35      0.35        17
     PREMIUM       0.15      0.15      0.15        13

    accuracy                           0.27        30
   macro avg       0.25      0.25      0.25        30
weighted avg       0.27      0.27      0.27        30


정확도(Accuracy): 0.27


- RFM 기준으로 확인했을 땐 세그멘테이션(등급 라벨링)이 정확도가 0.27로 잘못 되어 있는 것으로 확인할 수 있다. 

In [22]:
# 데이터셋 추출
rfm_df.rename(columns={'RFM_Segment3': 'Segmentation'}, inplace=True) # 세그먼트 컬럼명 변경
rfm_df.to_csv('rfm_segmentation_jhy.csv', index=False) # 데이터셋 추출

temp_df = pd.read_csv('rfm_segmentation_jhy.csv')
temp_df.head()

,prev_grade,last_order_date,Recency,Frequency,Monetary,R_Score,F_Score,M_Score,RFM_Score,RFM_Segment1,RFM_Segment2,Segmentation,RFM_Group
0,REGULAR,2024-05-16,3,9,3522000,4,4,4,12,Platinum,Vip,최우수고객,PREMIUM
1,PREMIUM,2024-03-19,61,3,2297000,2,1,3,6,Bronze,General,일반,REGULAR
2,REGULAR,2024-01-23,117,5,1209000,1,2,2,5,Bronze,General,일반,REGULAR
3,REGULAR,2024-03-31,49,7,5592000,2,3,4,9,Gold,At_Risk,이탈위험,PREMIUM
4,REGULAR,2024-04-16,33,6,6443000,3,2,4,9,Gold,Core,핵심고객,PREMIUM


#### 최종 결론
1. RFM score 기반으로 고객의 구매 패턴 예측을 위한 세그멘테이션을 진행하면 패턴을 명확하게 확인할 수 있다.
2. 사용한 데이터의 회원 수가 다소 적어서(30명) 8개의 기준으로 세그멘테이션을 처음 진행했으나, 절대적인 데이터 수가 부족해서 적절하지 않았고,
3. 최종적으로 최우수고객(VIP)를 제외한 총 5개 그룹(비활성, 이탈위험, 일반, 잠재, 최우수, 핵심 고객)으로 확인할 수 있다. 
4. 기업 입장에선 많은 고객을 관리할 때 해당 그룹에 필요한 마케팅과 전략을 데이터를 통해 맞춤형으로 접근해서 고객의 구매 유도를 이끌어 낼 수 있을 것이다.
    - VIP 고객에겐 R, F, M 지표 모두 유지하기 위한 특별 상품 + 퀵 배송 제공, 이탈 위험 고객에겐 앱 알림 등을 통한 프로모션으로 재구매 유도 등 
